# SENet

<img src="https://i.imgur.com/3xGwreb.png" width=1000/>

- [source paper](https://arxiv.org/abs/1709.01507)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import datasets, layers, Model, Sequential, losses

In [ ]:
(x_train, y_train), (x_test ,y_test) = datasets.mnist.load_data()

x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)
print(f'x_train shape:{x_train.shape}')
print(f'x_test shape:{x_test.shape}')
print('----------')

x_train = tf.repeat(x_train, 3, axis=3)
x_test = tf.repeat(x_test, 3, axis=3)
print(f'x_train shape:{x_train.shape}')
print(f'x_test shape:{x_test.shape}')
print('----------')

x_val = x_train[int(x_train.shape[0]*0.8):,:,:,:]
y_val = y_train[int(y_train.shape[0]*0.8):]
x_train = x_train[:int(x_train.shape[0]*0.8),:,:,:]
y_train = y_train[:int(y_train.shape[0]*0.8)]
print(f'x_train shape:{x_train.shape}, x_val shape:{x_val.shape}')
print(f'y_train shape:{y_train.shape}, y_val shape:{y_val.shape}')

x_train shape:(60000, 28, 28, 1)
x_test shape:(10000, 28, 28, 1)
----------
x_train shape:(60000, 28, 28, 3)
x_test shape:(10000, 28, 28, 3)
----------
x_train shape:(48000, 28, 28, 3), x_val shape:(12000, 28, 28, 3)
y_train shape:(48000,), y_val shape:(12000,)


In [ ]:
labels_num = 10
batch_size = 4

<img src="https://i.imgur.com/mvqWU9g.png" width=1000/>

- [source paper](https://arxiv.org/abs/1709.01507)

* ## API function

In [ ]:
def add_conv(x, filters_num, kernel_size):
  x = layers.Conv2D(filters_num, (kernel_size, kernel_size), strides=1, padding = 'same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)
  x = layers.MaxPooling2D((2, 2), strides=1, padding = 'same')(x)
  return x

filters_num = 64

inputs = layers.Input(shape=x_train.shape[1:])
x = layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=x_train.shape[1:])(inputs)
conv_1 = add_conv(x, filters_num, 7)
conv_2 = add_conv(conv_1, filters_num, 3)

squeeze = layers.GlobalAveragePooling2D()(conv_2)
squeeze = layers.Reshape((1, 1, filters_num))(squeeze)
excitation = layers.Dense(filters_num, activation = 'relu')(squeeze)
excitation = layers.Dense(filters_num, activation = 'sigmoid')(excitation)

scale = conv_2 * excitation
scale = layers.GlobalAveragePooling2D()(scale)
outputs = layers.Dense(labels_num)(scale)

In [ ]:
SENet_model = Model(inputs=inputs, outputs=outputs)

In [ ]:
SENet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 3)]  0           []                               
                                                                                                  
 resizing (Resizing)            (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 64  9472        ['resizing[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 64  256        ['conv2d[0][0]']             

In [ ]:
inputs = np.ones((batch_size, x_train.shape[1], x_train.shape[2], 3), dtype=np.float32)
SENet_model(inputs).shape

TensorShape([4, 10])

In [ ]:
SENet_model(inputs)

<tf.Tensor: shape=(4, 10), dtype=float32, numpy=
array([[-0.02457043, -0.0495788 ,  0.24664153, -0.00211908, -0.11318807,
        -0.07496573, -0.07785393, -0.07045375, -0.11746019, -0.03302393],
       [-0.02457043, -0.0495788 ,  0.24664153, -0.00211908, -0.11318807,
        -0.07496573, -0.07785393, -0.07045375, -0.11746019, -0.03302393],
       [-0.02457043, -0.0495788 ,  0.24664153, -0.00211908, -0.11318807,
        -0.07496573, -0.07785393, -0.07045375, -0.11746019, -0.03302393],
       [-0.02457043, -0.0495788 ,  0.24664153, -0.00211908, -0.11318807,
        -0.07496573, -0.07785393, -0.07045375, -0.11746019, -0.03302393]],
      dtype=float32)>

* ## OOP method

In [ ]:
class SEBlock(layers.Layer):
  def __init__(self, filter_nums):
    super(SEBlock, self).__init__()

    self.gap = layers.GlobalAveragePooling2D()
    self.squeeze_reshape = layers.Reshape((1, 1, filters_num))
    self.dense_1 = layers.Dense(filters_num, activation = 'relu')
    self.dense_2 = layers.Dense(filters_num, activation = 'sigmoid')
    self.multiply = tf.keras.layers.Multiply()

  def call(self, inputs, training = None):
    squeeze = self.gap(inputs)
    squeeze = self.squeeze_reshape(squeeze)

    excitation = self.dense_1(squeeze)
    excitation = self.dense_2(excitation)
    scale = inputs * excitation
    return scale

In [ ]:
class SENet(Model):
  def __init__(self, input_shape, filters_num, labels_num = 10):
    super(SENet, self).__init__()

    self.conv_1 = Sequential([layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=input_shape),
                  layers.Conv2D(filters_num, (7, 7), strides=1, padding = 'same'),
                  layers.BatchNormalization(),
                  layers.Activation('relu'),
                  layers.MaxPooling2D((2, 2), strides=1, padding = 'same')])

    self.conv_2 = Sequential([layers.Conv2D(filters_num, (3,3), strides=1, padding = 'same'),
                  layers.BatchNormalization(),
                  layers.Activation('relu'),
                  layers.MaxPooling2D((2, 2), strides=1, padding = 'same')])

    self.SEBlock = SEBlock(filters_num) 
    self.GAP = layers.GlobalAveragePooling2D()                
    self.classifier = layers.Dense(labels_num)    

  def call(self, inputs, training=None):
    x = self.conv_1(inputs)
    x = self.conv_2(x)
    x = self.SEBlock(x)
    x = self.GAP(x)
    x = self.classifier(x)
    return x

In [ ]:
SENet_model = SENet(input_shape = x_train.shape[1:], filters_num = 64)

In [ ]:
SENet_model.build((None, x_train.shape[1], x_train.shape[2], x_train.shape[3]))
SENet_model.summary()

Model: "se_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 224, 224, 64)      9728      
                                                                 
 sequential_1 (Sequential)   (None, 224, 224, 64)      37184     
                                                                 
 se_block (SEBlock)          multiple                  8320      
                                                                 
 global_average_pooling2d_3   multiple                 0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_5 (Dense)             multiple                  650       
                                                                 
Total params: 55,882
Trainable params: 55,626
Non-trainable params: 256
______________________________________________________

In [ ]:
inputs = np.ones((batch_size, x_train.shape[1], x_train.shape[2], 3), dtype=np.float32)
SENet_model(inputs).shape

TensorShape([4, 10])

In [ ]:
SENet_model(inputs)

<tf.Tensor: shape=(4, 10), dtype=float32, numpy=
array([[ 0.27074897,  0.0410268 ,  0.06213444, -0.07577617, -0.04352984,
        -0.01068289, -0.10910437, -0.03467438,  0.09753961,  0.0194334 ],
       [ 0.27074897,  0.0410268 ,  0.06213444, -0.07577617, -0.04352984,
        -0.01068289, -0.10910437, -0.03467438,  0.09753961,  0.0194334 ],
       [ 0.27074897,  0.0410268 ,  0.06213444, -0.07577617, -0.04352984,
        -0.01068289, -0.10910437, -0.03467438,  0.09753961,  0.0194334 ],
       [ 0.27074897,  0.0410268 ,  0.06213444, -0.07577617, -0.04352984,
        -0.01068289, -0.10910437, -0.03467438,  0.09753961,  0.0194334 ]],
      dtype=float32)>